In [1]:
pip install torch torchvision pillow tkinter ultralytics


Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement tkinter (from versions: none)
ERROR: No matching distribution found for tkinter


In [8]:
pip install ultralytics


Note: you may need to restart the kernel to use updated packages.


In [2]:
# Install dependencies
!pip install pyyaml==5.1
!pip install torch torchvision opencv-python

# Install Detectron2
!pip install git+https://github.com/facebookresearch/detectron2.git


  Cloning https://github.com/facebookresearch/detectron2.git to c:\users\linam\appdata\local\temp\pip-req-build-1j4i5fla


  ERROR: Error [WinError 2] The system cannot find the file specified while executing command git version
ERROR: Cannot find command 'git' - do you have 'git' installed and in your PATH?


In [3]:
!pip install torch torchvision opencv-python pyyaml==5.1


In [4]:
!pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu102/torch1.8/index.html


Looking in links: https://dl.fbaipublicfiles.com/detectron2/wheels/cu102/torch1.8/index.html


ERROR: Could not find a version that satisfies the requirement detectron2 (from versions: none)
ERROR: No matching distribution found for detectron2


In [7]:
!pip install torch torchvision opencv-python pyyaml==5.1


In [4]:

# note that the gui only works on jupyter and im having problems installing the other model so i will comment the implementation of the detectron model
import tkinter as tk
from tkinter import filedialog, Text
from PIL import Image, ImageTk
import torch
import torchvision.transforms as transforms
from ultralytics import YOLO
"""
import detectron2
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.model_zoo import model_zoo"""
import cv2
import numpy as np

class ObjectDetectionApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Object Detection App")

        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        
        self.image_path = tk.StringVar()
        self.model_path = tk.StringVar()

        tk.Label(root, text="Image Path:").pack()
        self.image_entry = tk.Entry(root, textvariable=self.image_path, width=50)
        self.image_entry.pack()
        
        tk.Button(root, text="Browse Image", command=self.browse_image).pack()

        tk.Label(root, text="Model Path:").pack()
        self.model_entry = tk.Entry(root, textvariable=self.model_path, width=50)
        self.model_entry.pack()
        
        tk.Button(root, text="Browse Model", command=self.browse_model).pack()

        tk.Label(root, text="Model Type:").pack()
        self.model_type = tk.StringVar(value="YOLOv8")
        tk.Radiobutton(root, text="YOLOv8", variable=self.model_type, value="YOLOv8").pack(anchor=tk.W)
        tk.Radiobutton(root, text="Detectron2", variable=self.model_type, value="Detectron2").pack(anchor=tk.W)

        self.image_label = tk.Label(root)
        self.image_label.pack()

        tk.Button(root, text="Detect", command=self.predict).pack()

    def browse_image(self):
        file_path = filedialog.askopenfilename()
        self.image_path.set(file_path)
        self.load_image(file_path)

    def browse_model(self):
        file_path = filedialog.askopenfilename()
        self.model_path.set(file_path)

    def load_image(self, file_path):
        image = Image.open(file_path)
        image = image.resize((600, 400), Image.LANCZOS)
        photo = ImageTk.PhotoImage(image)
        self.image_label.config(image=photo)
        self.image_label.image = photo

    def load_model(self, model_path, model_type):
        if model_type == "YOLOv8":
            self.model = YOLO(model_path)
        
        elif model_type == "Detectron2":
            cfg = get_cfg()
            cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
            cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5
            cfg.MODEL.WEIGHTS = model_path
            self.model = DefaultPredictor(cfg)

    def predict(self):
        image_path = self.image_path.get()
        model_path = self.model_path.get()
        model_type = self.model_type.get()

        if not image_path or not model_path:
            print("Please load an image and a model first.")
            return

        self.load_model(model_path, model_type)
        
        image = Image.open(image_path)
        image_cv = cv2.cvtColor(np.array(image), cv2.COLOR_RGB2BGR)

        if model_type == "YOLOv8":
            results = self.model(image_cv)
            result_img = results[0].plot()
        elif model_type == "Detectron2":
            outputs = self.model(image_cv)
            v = detectron2.utils.visualizer.Visualizer(image_cv[:, :, ::-1], scale=1.2)
            out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
            result_img = out.get_image()[:, :, ::-1]
        
        result_image = Image.fromarray(result_img)
        result_image = result_image.resize((600, 400), Image.LANCZOS)
        result_photo = ImageTk.PhotoImage(result_image)
        self.image_label.config(image=result_photo)
        self.image_label.image = result_photo

if __name__ == "__main__":
    root = tk.Tk()
    app = ObjectDetectionApp(root)
    root.mainloop()
